# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import requests
import json
import os
import gmaps
import gmaps.datasets

# Import display function
from IPython.display import display

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Path to cities.csv from Part I
cities_data_path = "../WeatherPy/output_data/cities.csv"

# Read the mouse data and the study results
cities_weather_data = pd.read_csv(cities_data_path)

# Drop the duplicate index column resulting from the CSV import
cities_weather_data.drop(columns=["Unnamed: 0"], inplace = True)

# Display a preview of the DataFrame
display(cities_weather_data)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Yar-Sale,66.83,70.83,22.41,95,6,8.95,RU
1,Wenling,28.37,121.36,66.00,85,100,9.48,CN
2,Birjand,32.87,59.22,73.40,5,0,9.17,IR
3,Tessalit,20.20,1.01,97.95,10,0,10.07,ML
4,Rikitea,-23.12,-134.97,74.07,81,54,21.05,PF
...,...,...,...,...,...,...,...,...
575,Noumea,-22.28,166.46,60.80,87,0,6.93,NC
576,Port Moresby,-9.44,147.18,78.80,88,75,11.41,PG
577,Bang Lamung,13.05,100.93,81.00,84,100,4.14,TH
578,Pizarro,4.95,-77.37,80.20,79,100,4.54,CO


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [106]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

 # Create a Heatmap layer with defined width and height
fig = gmaps.figure(layout = {'width': '900px',  'height': '600px'})

# Extract list of city humidities and convert to list for use as weights in heatmap
humidity_list = cities_weather_data["Humidity"].tolist()

# Find maximum humidity in data set for set as max intensity in heatmap
max_humidity = cities_weather_data["Humidity"].max()

# Construct heat layer
heat_layer = gmaps.heatmap_layer(cities_weather_data[["Lat", "Lng"]], humidity_list, 
                                 dissipating=False, max_intensity=float(max_humidity),
                                 point_radius = 1)
# Add heat layer to map
fig.add_layer(heat_layer)

# Display the map with heat layer
fig



Figure(layout=FigureLayout(height='600px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
# Select data fitting ideal weather conditions
# Max temp should be between 70 and 80, wind speed should less than 10 mph and cloudiness should be 0
vacation_cities_data = cities_weather_data.loc[(cities_weather_data["Max Temp"] > 70) & (cities_weather_data["Max Temp"] <  80) & (cities_weather_data["Wind Speed"] <  10) & (cities_weather_data["Cloudiness"] ==  0)]

# Drop any rows with null values
hotel_df = vacation_cities_data.dropna()

# Display DataFrame of list of cities and their weather data matching criteria
display(hotel_df)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
2,Birjand,32.87,59.22,73.40,5,0,9.17,IR
63,Vihari,30.03,72.35,78.94,16,0,2.37,PK
111,Hasan Abdal,30.43,72.70,79.63,15,0,3.56,PK
180,Chuy,-33.70,-53.46,73.69,60,0,6.38,UY
290,Emerald,-23.53,148.17,73.40,56,0,5.82,AU
413,Rio Grande,-32.03,-52.10,79.00,65,0,4.00,BR
440,Santa Vitoria Do Palmar,-33.52,-53.37,73.76,58,0,6.31,BR
506,Tura,25.52,90.22,75.09,91,0,2.46,RU
543,Taft,31.75,54.21,73.40,6,0,4.70,IR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
# Find the first hotel within 5000 metres of each city in the list

# Build base URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up intial parameters where target_type = lodging looks for hotels
# Hotel must be within 5000 metres of city
# Results should be returned in English
target_type = "lodging"
radius = 5000
language = "en-AU"

# Extract latitudes and longitudes of cities and convert to lists
latitude_list = hotel_df["Lat"].tolist()
longitude_list = hotel_df["Lng"].tolist()

# Initialise empty list to store hotels for vacation
vacation_hotels = []

# Loop through latitudes and longitudes list looking for hotel
for i in range(0, len(latitude_list)):
 
    
    # Set the coords parameter from the lists
    coords = f"{latitude_list[i]}, {longitude_list[i]}"


    # Fill out params dict with required information
    params = {
                "location": coords,
                "types": target_type,
                "radius": radius,
                "language" : language,
                    "key": g_key
            }

    # Run request to Google Maps API
    response = requests.get(base_url, params)

    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    # print(response.url)

    # Store JSON response in hotels variable
    hotels = response.json()

    # Print the JSON (pretty printed)
    # print(json.dumps(hotels, indent=4, sort_keys=True))

    # Append the hotel name to the list if one has been found
    try:
        
        vacation_hotels.append(hotels["results"][0]["name"])
    
    # Otherwise print message and append empty string to list
    except:
            
        # Display message indicating the hotel has been skipped
        print(f"Hotel not found. Skipping...")
        vacation_hotels.append("")
        
# Append hotel names as column to hotel_df DataFrme
hotel_df["Hotel Name"] = vacation_hotels

# Display DataFrame with names of hotels
display(hotel_df)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
2,Birjand,32.87,59.22,73.40,5,0,9.17,IR,Birjand Tourist Hotel
63,Vihari,30.03,72.35,78.94,16,0,2.37,PK,Al Shafi
111,Hasan Abdal,30.43,72.70,79.63,15,0,3.56,PK,FM Grewal House
180,Chuy,-33.70,-53.46,73.69,60,0,6.38,UY,Nuevo Hotel Plaza
290,Emerald,-23.53,148.17,73.40,56,0,5.82,AU,The Irish Village
413,Rio Grande,-32.03,-52.10,79.00,65,0,4.00,BR,Hotel Atlântico Rio Grande
440,Santa Vitoria Do Palmar,-33.52,-53.37,73.76,58,0,6.31,BR,Hostel Comodoro
506,Tura,25.52,90.22,75.09,91,0,2.46,RU,Hotel RIKMAN Continental
543,Taft,31.75,54.21,73.40,6,0,4.70,IR,بوم گردی حاجی زینل تفت


In [100]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [107]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
# Set info_box_content parameter as defined in previous cell
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='900px'))